In [6]:
import onnx
model = onnx.load(r"D:\OneDrive - Delft University of Technology\Master\Master_Thesis_Project\facedetect-finn-main\rebuild\tinyyolo-v0.2\tinyyolo-20210831.onnx")

In [ ]:
for node in model.graph.node:
    if node.name == "Conv_7":
        node.name